In [1]:
# Author : Arthur Prigent
# Email: aprigent@geomar.de

In [2]:
from load_librairies import *
import xscale.signal.fitting as xsf

dir_hist_sst = '/Volumes/Arthur_disk/Thesis_GEOMAR/Data/CMIP5/hist/tos/atl3/'
dir_hist_uas = '/Volumes/Arthur_disk/Thesis_GEOMAR/Data/CMIP5/hist/uas/atl4/'

dir_rcp8_sst = '/Volumes/Arthur_disk/Thesis_GEOMAR/Data/CMIP5/rcp85/tos/atl3/'
dir_rcp8_uas = '/Volumes/Arthur_disk/Thesis_GEOMAR/Data/CMIP5/rcp85/uas/atl4/'


path_fig = '/Users/aprigent/Documents/Thesis_GEOMAR/Projects/weakened_sst_variability_CMIP5/figures/'

import warnings
warnings.filterwarnings('ignore')


lon_min = -20
lon_max = 0
lat_min = -3
lat_max = 3

period_str_1 = 1982
period_end_2 = 1999

def is_jja(month):
    return (month >= 6) & (month <= 8)
def is_mjj(month):
    return (month >= 5) & (month <= 7)

def nandetrend(y):
    ''' Remove the linear trend from the data '''
    x = np.arange(0,y.shape[0],1)
    
    not_nan_ind = ~np.isnan(y)
    m, b, r_val, p_val, std_err = stats.linregress(x[not_nan_ind],np.array(y)[not_nan_ind])
    
    
    y_detrended_tmp= np.array(y) - m*x
    
    y_detrended  = xr.Dataset({'detrended': (['time'],y_detrended_tmp)}
                       ,coords={'time':y.time[:]})
    return y_detrended

# U wind

In [3]:
uwnd = xr.open_dataset('/Users/aprigent/Documents/Thesis_GEOMAR/Data/ERA_INTERIM/ERA_interim_uv_1979_2018_1x1.nc')

uwnd_12_tmp = uwnd.u10.sel(time = slice(datetime(period_str_1, 1, 1), datetime(period_end_2, 12, 31)))

uwnd_12_tmp = xr.concat([uwnd_12_tmp[:, :, 180:], uwnd_12_tmp[:, :, :180]], dim='longitude')
uwnd_12_tmp.coords['longitude'] = (uwnd_12_tmp.coords['longitude'] + 180) % 360 - 180

uwnd_12 = uwnd_12_tmp.chunk(chunks=None)
uwnd_12 = xsf.detrend(uwnd_12,dim='time',type='linear')

uwnd_12_ano,_ = Atools.ano_norm_t(uwnd_12.load())



In [4]:


uwnd_12_ano_Watl = uwnd_12_ano.where((uwnd_12_ano.longitude>=-40)&
                                                         (uwnd_12_ano.longitude<=-20)&
                                                         (uwnd_12_ano.latitude<=3)&
                                                         (uwnd_12_ano.latitude>=-3),
                                      drop=True).mean(dim='longitude').mean(dim='latitude')

# Z20

In [5]:
dir_data = '/Users/aprigent/Documents/Thesis_GEOMAR/Data/'
data     = dir_data + 'ORA-S4/'


file_sst = 'global_SST_ORA_S4_1958_2017_montlhy.nc'
nc = xr.open_mfdataset(data+file_sst)
lon_oras = np.arange(-60,20,1)
lat_oras = np.arange(-30,30,1)
time_oras = nc.TIME
#sst_oras = nc.THETAO



dir_data = '/Users/aprigent/Documents/Thesis_GEOMAR/Data/'
data = dir_data + 'ORA-S4/'
file = 'z20_depth_ORAS4_jan1958_dec_2017.nc'
#nc   = Dataset(data+file)
nc        = xr.open_mfdataset(data+file)
z20_tmp = nc.Z20
z20_tmp['time'] = np.array(time_oras)


z20_tmp = z20_tmp.sel(time = slice(datetime(period_str_1, 1, 1), datetime(period_end_2, 12, 31)))

#print(nc.variables.keys())
## z23 data from jan 1958 to dec 2017.
#z20_tmp = nc.variables['Z20'][:696,:,:] # taking the data from  jan 1958 to dec 2015 because of the wind data
#nc.close()
z20tmp_2=np.ones((np.shape(z20_tmp)))*np.nan
#


for i in range(z20tmp_2.shape[1]):
    for j in range(z20tmp_2.shape[2]):
        if np.isfinite(z20_tmp[:,i,j]).all()==True:
            z20tmp_2[:,i,j] = sc.detrend(z20_tmp[:,i,j],type='linear')

X,Y = np.meshgrid(lon_oras,lat_oras) 
#
#
z20  = xr.Dataset({'z20': (['time','y','x'],z20tmp_2)}
                   ,coords={'time':np.array(z20_tmp.time),'lat':(['y','x'],np.array(Y)),'lon':(['y','x'],np.array(X))})




In [6]:
z20_12 = z20.sel(time = slice(datetime(period_str_1, 1, 1), datetime(period_end_2, 12, 31)))

z20_12_atl3 = z20_12.where((z20_12.lon>=-20)&
                                                         (z20_12.lon<=0)&
                                                         (z20_12.lat<=3)&
                                                         (z20_12.lat>=-3),
                                   drop=True).mean(dim='y').mean(dim='x')


z20_12_atl4 = z20_12.where((z20_12.lon>=-40)&
                                                         (z20_12.lon<=-20)&
                                                         (z20_12.lat<=3)&
                                                         (z20_12.lat>=-3),
                                 drop=True).mean(dim='y').mean(dim='x')


In [7]:
z20_12_ano_atl3,_ = Atools.ano_norm_t(z20_12_atl3.z20)

In [8]:
thermo_slope_12 = z20_12_atl3.z20 - z20_12_atl4.z20

In [9]:
thermo_slope_12_ano,_ = Atools.ano_norm_t(thermo_slope_12)

# SST

In [10]:
# 1981-2019
## Load the data ##
nc = xr.open_dataset('/Users/aprigent/Documents/Thesis_GEOMAR/Data/OI_SST_v2/oi.sst.v2.monmean.nc')
sst_oiss = nc.sst[:]
sst_oiss = xr.concat([sst_oiss[:, :, 720:], sst_oiss[:, :, :720]], dim='lon')
sst_oiss.coords['lon'] = (sst_oiss.coords['lon'] + 180) % 360 - 180

## Take the period 1982/01-2017/12 ##
sst_oiss = sst_oiss.sel(time=slice(datetime(period_str_1, 1, 1), datetime(period_end_2, 12, 31)))

## Detrend the data ##
sst_oiss = sst_oiss.chunk(chunks=None)
sst_oiss = xsf.detrend(sst_oiss,dim='time',type='linear')


sst_oiss_atl3 = Atools.data_sub(sst_oiss,-20,0,-3,3).mean(dim='lon').mean(dim='lat')
ssta_oiss_atl3,_ = Atools.ano_norm_t(sst_oiss_atl3.load())

# take JJA anomalies

In [11]:
sst_12_ano_atl3_jja = ssta_oiss_atl3.sel(time=is_jja(ssta_oiss_atl3['time.month']))
uwnd_12_ano_Watl_jja =  uwnd_12_ano_Watl.sel(time=is_jja(uwnd_12_ano_Watl['time.month']))
thermo_slope_12_jja = thermo_slope_12.sel(time=is_jja(thermo_slope_12['time.month']))
z20_12_ano_atl3_jja  = z20_12_ano_atl3.sel(time=is_jja(z20_12_ano_atl3['time.month']))

In [12]:
sst_12_ano_atl3_mjj = ssta_oiss_atl3.sel(time=is_mjj(ssta_oiss_atl3['time.month']))
z20_12_ano_atl3_mjj  = z20_12_ano_atl3.sel(time=is_mjj(z20_12_ano_atl3['time.month']))

# BF1

In [13]:
alpha_12_bf1,_, r_alpha_12_bf1, p_value_12_bf1, std_12_bf1    = stats.linregress(
                                                                       sst_12_ano_atl3_jja,
                                                                       uwnd_12_ano_Watl_jja) 

In [14]:
print('BF1 period 12:', np.round(alpha_12_bf1,2))

BF1 period 12: 0.61


# BF2

In [15]:
alpha_12_bf2,_, r_alpha_12_bf2, p_value_12_bf2, std_12_bf2    = stats.linregress(
                                                                       uwnd_12_ano_Watl_jja,
                                                                       thermo_slope_12_jja) 
print('BF2 period 12:', np.round(alpha_12_bf2,2))

BF2 period 12: 10.77


# BF3

In [16]:



alpha_12_bf3,_,r_alpha_12_bf3,_,std_12_bf3    = stats.linregress(z20_12_ano_atl3_jja,
                                                              sst_12_ano_atl3_jja) 

In [17]:
print('BF3 period 12:', np.round(alpha_12_bf3,3))


BF3 period 12: 0.083


# ORA-S4 SST

In [18]:
data     = dir_data + 'ORA-S4/'
file_sst = 'global_SST_ORA_S4_1958_2017_montlhy.nc'
print('Loading the file :',data+file_sst)
nc = xr.open_mfdataset(data+file_sst)


SST_tmp= nc.THETAO[:,0,:,:]
SST_tmp =SST_tmp.sel(TIME = slice(datetime(1958, 1, 1), datetime(2017, 12, 31)))
tmp = xr.concat([SST_tmp[:,:,180:],SST_tmp[:,:,:180]],dim = 'LON') 
tmp.coords['LON'] = (tmp.coords['LON'] + 180)%360 - 180

lon = tmp.LON[:]
lat = tmp.LAT[:]
time_oras = tmp.TIME[:]
tmp = tmp.load()
sst_oras_tmp = tmp
 



sst_oras  = xr.Dataset({'sst': (['time','lat','lon'],sst_oras_tmp)}
                   ,coords={'time':np.array(time_oras),'lat':(np.array(lat)),'lon':(np.array(lon))})


Loading the file : /Users/aprigent/Documents/Thesis_GEOMAR/Data/ORA-S4/global_SST_ORA_S4_1958_2017_montlhy.nc


In [19]:
sst_oras_12_tmp = sst_oras.sel(
    time=slice(datetime(period_str_1, 1, 1), datetime(period_end_2, 12, 31)))
sst_oras_12_test = sst_oras_12_tmp.sst.chunk(chunks=None)
sst_oras_12 = xsf.detrend(sst_oras_12_test,dim='time',type='linear')

sst_oras_12_ano, _ = Atools.ano_norm_t(sst_oras_12.load())

sst_oras_12_ano_atl3 = Atools.data_sub(sst_oras_12_ano,-20,0,-3,3).mean(dim='lon').mean(dim='lat')

In [20]:
sst_12_ano_atl3_jja = sst_oras_12_ano_atl3.sel(time=is_jja(sst_oras_12_ano_atl3['time.month']))
z20_12_ano_atl3_jja  = z20_12_ano_atl3.sel(time=is_jja(z20_12_ano_atl3['time.month']))

In [21]:

alpha_12_bf3,_,r_alpha_12_bf3,_,std_12_bf3    = stats.linregress(z20_12_ano_atl3_jja,
                                                              sst_12_ano_atl3_jja) 

In [23]:
print('BF3 JJA ORA-S4',alpha_12_bf3)

BF3 JJA ORA-S4 0.09983830288402684
